# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

from pyopia.classify import Classify
import exampledata
import pyopia.io
from pyopia.pipeline import Pipeline, Common
from pyopia.instrument.silcam import SilCamLoad, ImagePrep
from pyopia.process import CalculateStats

In [3]:
filename = exampledata.get_example_silc_image()


In [4]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/test'
model_path = exampledata.get_example_model()
threshold = 0.98

steps = {'common': Common(),
         'load': SilCamLoad(filename),
         'classifier': Classify(model_path=model_path),
         'imageprep': ImagePrep(),
         'statextract': CalculateStats(threshold=threshold),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps)


In [5]:
stats = processing_pipeline.run()

segment
clean
2.2% saturation
measure
  121 particles found
WARNING. exportparticles temporarily modified for 2-d images without color!
EXTRACTING 121 IMAGES from 121


In [6]:

# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

Modified:
    2022-11-05 22:14:10.835376
Pipeline steps:
    
1) Step: common
   Type: <class 'pyopia.pipeline.Common'>
   Vars: {}
2) Step: load
   Type: <class 'pyopia.instrument.silcam.SilCamLoad'>
   Vars: {'filename': 'D20181101T142731.838206.silc'}
3) Step: classifier
   Type: <class 'pyopia.classify.Classify'>
   Vars: {'model_path': 'keras_model.h5', 'class_labels': Index(['oil', 'other', 'bubble', 'faecal_pellets', 'copepod', 'diatom_chain',
       'oily_gas'],
      dtype='object'), 'model': <keras.engine.sequential.Sequential object at 0x00000218F6626B20>}
4) Step: imageprep
   Type: <class 'pyopia.instrument.silcam.ImagePrep'>
   Vars: {}
5) Step: statextract
   Type: <class 'pyopia.process.CalculateStats'>
   Vars: {'minimum_area': 12, 'threshold': 0.98, 'real_time_stats': False, 'max_coverage': 30, 'max_particles': 5000}
6) Step: output
   Type: <class 'pyopia.io.StatsH5'>
   Vars: {'datafilename': 'proc/test'}

PyOpia version:
    0.0.11


In [7]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))

stats header:  Index(['major_axis_length', 'minor_axis_length', 'equivalent_diameter',
       'solidity', 'minr', 'minc', 'maxr', 'maxc', 'probability_oil',
       'probability_other', 'probability_bubble', 'probability_faecal_pellets',
       'probability_copepod', 'probability_diatom_chain',
       'probability_oily_gas', 'export name', 'timestamp', 'saturation'],
      dtype='object')
Total number of particles:  1605
